In [2]:
# %load_ext autoreload
# %autoreload 2

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import re
import glob #for assembling files 
import wandb
import emoji

import sys
sys.path.insert(1, '../python')

from cb_preprocess import load_data, preprocess, tokenize, label_to_numeric
from nltk import ngrams
from nltk.corpus import stopwords
from collections import Counter

import preprocessor as pre
import fasttext as ft

from tokenizers import Tokenizer, normalizers
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import WordPiece as WP
from tokenizers.normalizers import NFD, StripAccents, Lowercase
from tokenizers.trainers import WordPieceTrainer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
##assemble dataset from 6 files in ../../data_wang
files = glob.glob('../../data/*.txt')

df_list = []
for file in files: 
    with open(file, 'r', encoding="UTF-8") as f: 
        tweets = f.readlines()
        label = file[file.find("8000")+len("8000"):file.find(".txt")]
        df_list.append(pd.DataFrame({"tweet_text": tweets, "label": np.repeat(label, len(tweets))}))

pd.concat(df_list).to_csv("../../data/wang_assembled.csv", index=True)

In [5]:
df = load_data(path = "../../data/wang_assembled.csv")
df["y"] = label_to_numeric(df)
df["tweet_text"] = preprocess(df.tweet_text)
df["tokens"] = [" ".join(tokens) for tokens in tokenize(df.tweet_text)]

df

,tweet_text,label,y,tokens
0,Here at home. Neighbors pick on my family and ...,age,0,here at home . neighbors pick on my family and...
1,Being bullied at school: High-achieving boys u...,age,0,being bullied at school : high - achieving boy...
2,There was a girl in my class in 6th grade who ...,age,0,there was a girl in my class in 6th grade who ...
3,He’s probably a white gay kid from some suburb...,age,0,he ’ s probably a white gay kid from some subu...
4,You are pushed ti resorting. Treating thr bull...,age,0,you are pushed ti resorting . treating thr bul...
...,...,...,...,...
7995,Can you imagine if Christians came together li...,religion,5,can you imagine if christians came together li...
7996,So how to support justice from the initial pro...,religion,5,so how to support justice from the initial pro...
7997,$RESERVED$ $MENTION$: If you harbour any doubt...,religion,5,$RESERVED$ $MENTION$ : if you harb ##our any d...
7998,"$MENTION$ $MENTION$ One thing about Muslims, t...",religion,5,"$MENTION$ $MENTION$ one thing about muslims , ..."


In [6]:
class LanguageIdentification:

    def __init__(self):
        pretrained_lang_model = "../fasttext_binaries/lid.176.bin"
        self.model = ft.load_model(pretrained_lang_model)

    def predict_lang(self, text):
        predictions = self.model.predict(text, k=2) # returns top 2 matching languages
        return predictions

In [8]:
language_identifer = LanguageIdentification()
lang_predictions = df.tweet_text.apply(language_identifer.predict_lang)

In [9]:
df["lang"] = lang_predictions.apply(lambda x: x[0][0][-2:])

df.to_csv("../../data/wang_preprocessed.csv", index=True)
df.groupby("lang").count()

In [11]:
df

,tweet_text,label,y,tokens
lang,,,,
af,1,1,1,1
ar,1,1,1,1
bg,2,2,2,2
bk,2,2,2,2
bs,1,1,1,1
ca,1,1,1,1
cs,96,96,96,96
da,1,1,1,1
de,61,61,61,61


In [79]:
run = wandb.init(project='capstone')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ayushkumar (cyberbullying). Use `wandb login --relogin` to force relogin


In [84]:
artifact = wandb.Artifact("tweets", type="dataset")
artifact.add_dir("../../data_wang/")
wandb.log_artifact(artifact, aliases = ["preprocessed_tweets"])

wandb: Adding directory to artifact (.\..\..\data_wang)... Done. 0.1s


Error: You must call wandb.init() before wandb.log_artifact()

In [171]:
artifact = wandb.Artifact("wp_pretrained_tweet", type = "tokenizer")
artifact.add_file("../wp_tokenizer.json")
wandb.log_artifact(artifact)

In [173]:
table = wandb.Table(dataframe = df)
run.log({"data": table})

In [81]:
wandb.finish()

wandb: ERROR Control-C detected -- Run data was not synced
